In [9]:
from transformers import AutoTokenizer, BioGptModel
import torch

device = 'cuda:1'

tokenizer = AutoTokenizer.from_pretrained("microsoft/biogpt")
model = BioGptModel.from_pretrained("microsoft/biogpt").to(device)

inputs = tokenizer("Hello, my dog is cute", return_tensors="pt").to(device)
outputs = model(**inputs)

/opt/conda/envs/py_3.10/lib/python3.10/site-packages/transformers/models/biogpt/modeling_biogpt.py:330: UserWarning: 1Torch was not compiled with memory efficient attention. (Triggered internally at /var/lib/jenkins/pytorch/aten/src/ATen/native/transformers/hip/sdp_utils.cpp:505.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [12]:
outputs.last_hidden_state.shape

torch.Size([1, 10, 1024])

In [5]:
model

BioGptModel(
  (embed_tokens): BioGptScaledWordEmbedding(42384, 1024, padding_idx=1)
  (embed_positions): BioGptLearnedPositionalEmbedding(1026, 1024)
  (layers): ModuleList(
    (0-23): 24 x BioGptDecoderLayer(
      (self_attn): BioGptSdpaAttention(
        (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
        (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
        (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
        (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
      )
      (activation_fn): GELUActivation()
      (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (fc1): Linear(in_features=1024, out_features=4096, bias=True)
      (fc2): Linear(in_features=4096, out_features=1024, bias=True)
      (final_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    )
  )
  (layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
)

In [14]:
inputs.tokens

<bound method BatchEncoding.tokens of {'input_ids': tensor([[    2,   313,  3666,   399,     7, 10241,  4316,    21,   704,  9070]],
       device='cuda:1'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:1')}>

In [34]:
str_input = "Viruses, Riboviria, Pararnavirae, Artverviricota, Revtraviricetes, Ortervirales, Retroviridae, Orthoretrovirinae, Lentivirus, Primate lentivirus group, unclassified Primate lentivirus group, Human immunodeficiency virus"

taxa = [taxon.strip() for taxon in str_input.split(',')]
processed_text =" ".join(taxa)

str_input = processed_text
print(str_input)
inputs = tokenizer(str_input, return_tensors="pt", add_special_tokens=False)
input_ids = inputs.input_ids[0].tolist()
# print(input_ids)
for _ in range(len(input_ids)):
    print(input_ids[_], "\t", tokenizer.decode(input_ids[_]))


Viruses Riboviria Pararnavirae Artverviricota Revtraviricetes Ortervirales Retroviridae Orthoretrovirinae Lentivirus Primate lentivirus group unclassified Primate lentivirus group Human immunodeficiency virus
29386 	 Viruses
13699 	 Rib
33030 	 ovi
12347 	 ria
3587 	 Par
1162 	 ar
790 	 n
13583 	 avi
36460 	 rae
2897 	 Ar
765 	 t
3238 	 ver
8493 	 vir
1356 	 ic
15373 	 ota
12924 	 Rev
9499 	 tra
8493 	 vir
1356 	 ic
10474 	 etes
6530 	 Or
2543 	 ter
8493 	 vir
9545 	 ales
16218 	 Retro
22269 	 viridae
11523 	 Orth
5065 	 ore
5230 	 tro
8493 	 vir
14739 	 inae
237 	 L
5735 	 enti
350 	 virus
11584 	 Prim
786 	 ate
28524 	 lentivirus
64 	 group
446 	 un
1856 	 classified
11584 	 Prim
786 	 ate
28524 	 lentivirus
64 	 group
1969 	 Human
3317 	 immunodeficiency
350 	 virus


In [35]:
from transformers import AutoTokenizer, EsmModel
import torch

device = 'cuda:1' if torch.cuda.is_available() else 'cpu'
print(device)

print('allocated', torch.cuda.memory_allocated() / 1024**2)  # MB
print('cached', torch.cuda.memory_reserved() / 1024**2)      # MB

model = EsmModel.from_pretrained("facebook/esm2_t6_8M_UR50D")
# Force CUDA to sync
torch.cuda.synchronize()

print('allocated', torch.cuda.memory_allocated() / 1024**2)  # MB
print('cached', torch.cuda.memory_reserved() / 1024**2)      # MB

inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
outputs = model(**inputs)

last_hidden_states = outputs.last_hidden_state

Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t6_8M_UR50D and are newly initialized: ['esm.pooler.dense.bias', 'esm.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [36]:
inputs

{'input_ids': tensor([[ 0, 21,  3,  3,  3,  3,  3,  2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}

In [38]:
print(outputs.last_hidden_state.shape)
print(outputs.pooler_output.shape)

torch.Size([1, 8, 320])
torch.Size([1, 320])


In [4]:
import sys, os
from transformers import AutoTokenizer, EsmModel
import torch
import torch.nn as nn
import torch.nn.functional as F

sys.path.insert(0, '../dlp')

device = 'cuda:1' if torch.cuda.is_available() else 'cpu'
print(device)

tokenizer = AutoTokenizer.from_pretrained("facebook/esm2_t33_650M_UR50D")
def data_tensor_batch(b, max_seq_len):
    return tokenizer(
        [e['Sequence'] for e in b],
        return_tensors="pt", 
        padding='max_length', 
        truncation=True, 
        max_length=max_seq_len
    ).to(device)


class ESM2(nn.Module):
    def __init__(self,model_name, num_classes):
        super().__init__()
        self.esm = EsmModel.from_pretrained(model_name)
        # for param in self.esm.parameters():
        #     param.requires_grad = False
        self.layer1 = nn.Linear(1280, 512)
        self.relu = nn.ReLU()
        self.layer2 = nn.Linear(512, num_classes)
        
    def forward(self, x, attention_mask=None):
        outputs = self.esm(x, attention_mask=attention_mask).pooler_output
        outputs = self.layer1(outputs)
        outputs = self.relu(outputs)
        outputs = self.layer2(outputs)
        return outputs

cuda:1


In [7]:
import time
def eval_model(model, max_seq_len, num_val=10):
    model.eval()  # Set model to evaluation mode
    
    timing_stats = {
        'batch_times': [],
        'total_time': 0
    }
    
    total_start_time = time.time()
    
    for epoch in range(num_val):
        with torch.no_grad():
            batch_start_time = time.time()
    
            tensor_batch = data_tensor_batch(da.get_batch(), max_seq_len)
            outputs = model(tensor_batch['input_ids'], tensor_batch['attention_mask'])
    
            batch_end_time = time.time()
            
            batch_duration = batch_end_time - batch_start_time
            timing_stats['batch_times'].append(batch_duration)
            
            print(f"Batch {epoch + 1}/{num_val} took {batch_duration:.3f} seconds")
    
    total_end_time = time.time()
    total_duration = total_end_time - total_start_time
    timing_stats['total_time'] = total_duration
    
    # Calculate statistics
    avg_batch_time = sum(timing_stats['batch_times']) / len(timing_stats['batch_times'])
    print(f"\nEvaluation Summary:")
    print(f"Total time: {total_duration:.3f} seconds")
    print(f"Average batch time: {avg_batch_time:.3f} seconds")


def train_model(model, max_seq_len, num_train=10):
    model.train()  # Set model to evaluation mode
    
    timing_stats = {
        'batch_times': [],
        'total_time': 0
    }
    
    total_start_time = time.time()
    
    for epoch in range(num_train):
        batch_start_time = time.time()
    
        tensor_batch = data_tensor_batch(da.get_batch(), max_seq_len)
        outputs = model(tensor_batch['input_ids'], tensor_batch['attention_mask'])
    
        loss = criterion(outputs, torch.zeros((batch_size, )).to(device).long())
    
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        batch_end_time = time.time()
        batch_duration = batch_end_time - batch_start_time
        timing_stats['batch_times'].append(batch_duration)
        
        print(f"Batch {epoch + 1}/{num_train} took {batch_duration:.3f} seconds")
    
    total_end_time = time.time()
    total_duration = total_end_time - total_start_time
    timing_stats['total_time'] = total_duration
    
    # Calculate statistics
    avg_batch_time = sum(timing_stats['batch_times']) / len(timing_stats['batch_times'])
    print(f"\nEvaluation Summary:")
    print(f"Total time: {total_duration:.3f} seconds")
    print(f"Average batch time: {avg_batch_time:.3f} seconds")

In [10]:
model_name = "facebook/esm2_t33_650M_UR50D"
dataset_name = "corpus_200_500_random"
batch_size = 2
from data_access import PQDataAccess
da = PQDataAccess(f"/home/aac/Alireza/datasets/export_pqt_4_taxseq_new/{dataset_name}", batch_size)
max_seq_len = 1022

print('allocated', torch.cuda.memory_allocated() / 1024**2)  # MB
print('cached', torch.cuda.memory_reserved() / 1024**2)      # MB

model = ESM2(model_name, 1).to(device)
# Force CUDA to sync
torch.cuda.synchronize()

print('allocated', torch.cuda.memory_allocated() / 1024**2)  # MB
print('cached', torch.cuda.memory_reserved() / 1024**2)      # MB

trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total = sum(p.numel() for p in model.parameters())
print(f'Trainable parameters: {trainable/ 1e6} M')
print(f'Total parameters: {total/ 1e6} M')
# print(model)

optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=.001)
criterion = nn.CrossEntropyLoss()
# Cosine annealing with warm restarts
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
    optimizer,
    T_0=10,  # Initial restart interval
    T_mult=2,  # Multiply interval by 2 after each restart
    eta_min=1e-6  # Minimum learning rate
)

train_model(model, max_seq_len)
print("---------------------------------------")
eval_model(model, max_seq_len)

allocated 0.0
cached 0.0


Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t33_650M_UR50D and are newly initialized: ['esm.pooler.dense.bias', 'esm.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


allocated 0.0
cached 0.0
Trainable parameters: 653.010326 M
Total parameters: 653.010326 M
Batch 1/10 took 1.423 seconds
Batch 2/10 took 0.532 seconds
Batch 3/10 took 0.522 seconds
Batch 4/10 took 0.521 seconds
Batch 5/10 took 0.521 seconds
Batch 6/10 took 0.523 seconds
Batch 7/10 took 0.530 seconds
Batch 8/10 took 0.525 seconds
Batch 9/10 took 0.523 seconds
Batch 10/10 took 0.522 seconds

Evaluation Summary:
Total time: 6.144 seconds
Average batch time: 0.614 seconds
---------------------------------------
Batch 1/10 took 0.097 seconds
Batch 2/10 took 0.168 seconds
Batch 3/10 took 0.166 seconds
Batch 4/10 took 0.169 seconds
Batch 5/10 took 0.169 seconds
Batch 6/10 took 0.169 seconds
Batch 7/10 took 0.169 seconds
Batch 8/10 took 0.169 seconds
Batch 9/10 took 0.169 seconds
Batch 10/10 took 0.170 seconds

Evaluation Summary:
Total time: 1.616 seconds
Average batch time: 0.162 seconds
